In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# standard lib
import os, pwd, sys, json, yaml, atexit, tempfile, inspect
from pathlib import Path

# for data-science
import pandas as pd, numpy as np, quadfeather
from pyarrow import feather

# for plotting
import matplotlib as mpl, matplotlib.pyplot as plt, seaborn as sns

# for cellular-data
import scprep, scanpy as sc, anndata as ad

In [3]:
from featherplot.utils import MockSingleCellData, AnnDataProcessor, QuadFeatherRenamer
from featherplot.utils import SeriesToChannel, DataFrameToMetadata

from featherplot.utils import collapse_user
from featherplot.deepscatter import Tileset

In [4]:
adata = ad.read_h5ad(os.path.expanduser('~/Downloads/adata_s1.h5'))

In [5]:
adata

AnnData object with n_obs × n_vars = 41253 × 29275
    obs: 'barcodes', 'batch', 'timepoint', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mito', 'log1p_total_counts_mito', 'pct_counts_mito', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'log10_total_counts'
    var: 'ensembl_id', 'gene_symbol', 'feature_type', 'mito', 'ribo', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable_WT1', 'highly_variable_WT2', 'highly_variable_KO1', 'highly_variable_KO3', 'highly_variable'
    uns: 'batch_colors'
    obsm: 'X_pca', 'X_pca_hvg', 'X_phate', 'X_phate_hvg'
    layers: 'X_detected', 'X_magic', 'X_prenorm', 'X_scaled_normalised'

In [7]:
adata.obs.loc[:, 'conditions'] = adata.obs.batch

In [5]:
adata

AnnData object with n_obs × n_vars = 1000 × 100
    obs: 'barcodes', 'conditions'
    var: 'is_hvg'
    obsm: 'X_mock'
    layers: 'X_norm'

### HVGs

In [25]:
barcodes = pd.Series(adata.obs.index.to_numpy().astype(str), name='barcode')
subcells = barcodes.sample(20000)

allgenes = pd.Series(adata.var.gene_symbol.to_numpy(), name="genes")
hvggenes = pd.Series(adata.var[adata.var.highly_variable].gene_symbol.to_numpy(), name="hvg_genes")

hvg_idxs = hvggenes.map(lambda g: adata.var_names.tolist().index(g))

In [48]:
sdata = adata[subcells, hvg_idxs]
sdata.shape

(20000, 6274)

### Create Processor
> this will help us extract the embedding layer and the gene expression layer

In [49]:
pipe = AnnDataProcessor(sdata, 'X_phate_hvg', 'X_magic')

#### sidecars

Deepscatter calls additional columns `sidecars`, in our case those are the columns of gene expression. We place these values in `df_s`.

In [50]:
df_s = pipe.get_sidecars()
df_s.head()

gene_symbol,SAMD11,HES4,ISG15,ENSG00000242590,PUSL1,AURKAIP1,CCNL2,MRPL20-AS1,MRPL20,VWA1,...,SRY,RPS4Y1,ZFY,TBL1Y,USP9Y,DDX3Y,TMSB4Y,TTTY14,ENSG00000273748,ENSG00000276345
AGGCAGTGACCAAGAA_1_1,0.006149,0.167977,0.475078,0.024577,0.150066,0.912030,0.058636,0.094406,1.180757,0.016630,...,0.037425,1.451122,0.022790,0.085025,0.050572,0.060684,0.168853,0.740216,0.048749,0.219039
CAGCAGAAAGTGATTG_1_0,0.054882,0.141022,0.284296,0.034572,0.095772,0.927549,0.048393,0.093624,0.930962,0.032577,...,0.046954,1.436682,0.031922,0.053767,0.085013,0.053730,0.065288,0.477350,0.017662,0.150637
AAGCACAGAACCAGGC_0_1,0.013995,0.159398,0.379221,0.029823,0.149699,0.864707,0.092350,0.112568,1.096096,0.013355,...,0.043673,1.454330,0.025345,0.059932,0.060339,0.052731,0.117253,0.616852,0.044294,0.240017
AGCCATTCAGGTAATG_1_1,0.006551,0.108100,0.517338,0.037478,0.167430,0.993215,0.079535,0.096444,1.263467,0.021408,...,0.044514,1.526370,0.024822,0.118452,0.061080,0.066690,0.175099,0.822873,0.050498,0.255517
ATACACTGAGGCCCCG_1_0,0.014310,0.159075,0.296488,0.030551,0.126787,0.934317,0.059586,0.109673,1.068769,0.018880,...,0.049627,1.513220,0.020323,0.052966,0.056577,0.049664,0.081523,0.563833,0.034348,0.195569


#### points

If our gene expression features are called `sidecars`, then what is the embedding layer called? Well it is just the "points" of the plot, so we will store these values in `df_p`.

**NOTE**: we also store conditions with `df_p` as whatever is in this DataFrame will be loaded by `Deepscatter` automatically. 

In [51]:
df_p = pipe.get_embedding()
df_p = df_p.join(pipe.adata.obs.conditions)
df_p.head()

,PHATE_HVG_1,PHATE_HVG_2,PHATE_HVG_3,conditions
AGGCAGTGACCAAGAA_1_1,-0.019512,-0.001581,0.001367,WT2
CAGCAGAAAGTGATTG_1_0,-0.032605,0.002190,0.002802,WT1
AAGCACAGAACCAGGC_0_1,-0.025619,0.000695,0.002344,WT2
AGCCATTCAGGTAATG_1_1,-0.013609,-0.000721,0.000391,WT2
ATACACTGAGGCCCCG_1_0,-0.028058,0.000873,0.002455,WT1


#### Combined
Now we combine `df_p` (points + condition) with `df_s` ("sidecars" i.e. gene expression). This is necessary as for script later on where we need to add the sidecars to already the `quadfeather`-ed (tiled) point data. 

In [52]:
df_all = df_p.join(df_s)
df_all.head()

,PHATE_HVG_1,PHATE_HVG_2,PHATE_HVG_3,conditions,SAMD11,HES4,ISG15,ENSG00000242590,PUSL1,AURKAIP1,...,SRY,RPS4Y1,ZFY,TBL1Y,USP9Y,DDX3Y,TMSB4Y,TTTY14,ENSG00000273748,ENSG00000276345
AGGCAGTGACCAAGAA_1_1,-0.019512,-0.001581,0.001367,WT2,0.006149,0.167977,0.475078,0.024577,0.150066,0.912030,...,0.037425,1.451122,0.022790,0.085025,0.050572,0.060684,0.168853,0.740216,0.048749,0.219039
CAGCAGAAAGTGATTG_1_0,-0.032605,0.002190,0.002802,WT1,0.054882,0.141022,0.284296,0.034572,0.095772,0.927549,...,0.046954,1.436682,0.031922,0.053767,0.085013,0.053730,0.065288,0.477350,0.017662,0.150637
AAGCACAGAACCAGGC_0_1,-0.025619,0.000695,0.002344,WT2,0.013995,0.159398,0.379221,0.029823,0.149699,0.864707,...,0.043673,1.454330,0.025345,0.059932,0.060339,0.052731,0.117253,0.616852,0.044294,0.240017
AGCCATTCAGGTAATG_1_1,-0.013609,-0.000721,0.000391,WT2,0.006551,0.108100,0.517338,0.037478,0.167430,0.993215,...,0.044514,1.526370,0.024822,0.118452,0.061080,0.066690,0.175099,0.822873,0.050498,0.255517
ATACACTGAGGCCCCG_1_0,-0.028058,0.000873,0.002455,WT1,0.014310,0.159075,0.296488,0.030551,0.126787,0.934317,...,0.049627,1.513220,0.020323,0.052966,0.056577,0.049664,0.081523,0.563833,0.034348,0.195569


### QuadFeatherRenamer

Note: `quadfeather` and `deepscatter` are both under active development so things change all the time. At the moment `quadfeather` requires that `x` and `y` be in your DataFrame (it doesn't mind if `z` is there too). So this will handle the renaming of our columns.

In [104]:
qfr = QuadFeatherRenamer(df_all)

In [105]:
df_q, renamed = qfr.rename()
renamed

{'PHATE_HVG_1': 'x', 'PHATE_HVG_2': 'y', 'PHATE_HVG_3': 'z'}

### DataFrameToMetadata
 
`Deepscatter` is a really nice library; however, it also prefers to have its `plotAPI` method called with as much information as possible. This is a bit of a shame as it means that one you load your data with `deepscatter` you can't compute derived properties (e.g. domain of your data to scale the plot, check for what sidecars are availble, etc). 

The solution to this is simple. In order to have this information availble to us, we will just calculate it now (including which columns were renamed) and store it as metadata to use later

In [108]:
# NOTE: it is important to make sure that the index has a name!
df_q.index.name = 'barcodes'

#### unique columns

In [109]:
all_cols = df_q.columns.tolist()
uni_cols, col_cnts = np.unique(all_cols, return_counts=True)

for n, c in zip(uni_cols, col_cnts):
    if c == 1:
        continue

    i = 0
    for j, o in enumerate(all_cols):
        if o != n:
            continue
        all_cols[j] = f'{n}_{i}'
        i += 1

df_q.columns = all_cols

#### perpare metadata

In [111]:
d2m = DataFrameToMetadata(
    df_q, 
    include_index=True,
    embedding='x y z'.split(),
    alt_names={v:k for k,v in renamed.items()}
)

In [112]:
succ, fail = d2m.convert()
len(succ), len(fail)

(6279, 0)

In [113]:
meta = d2m.to_meta()

## Quadfeather Workflow

Now we can now run through thte `quadfeather` workflow right here in the notebook.

### 0) setup

In [115]:
# dump everything to downloads for easy access
outdir = os.path.expanduser('~/Downloads/seo_feather')
qf_dir = os.path.join(outdir, 'tiles')
if not os.path.isdir(qf_dir):
    os.makedirs(qf_dir)


p_file = os.path.join(outdir, 'points.parquet')
# NOTE: we never use s_file
# s_file = os.path.join(outdir, 'extras.parquet')
f_file = os.path.join(qf_dir, 'sidecars.feather')
m_file = os.path.join(outdir, 'meta.yml')


tile_size = 1000

### 1) create tiles

In [116]:
d2m.df.drop(columns=d2m.sidecars).to_parquet(p_file)
# d2m.df.drop(columns=d2m.embedding).to_parquet(s_file)

In [117]:
!quadfeather --files {p_file} \
             --tile_size {tile_size} \
             --destination {qf_dir}

### 2) make single file

In [119]:
feather.write_feather(d2m.df.drop(columns=d2m.embedding), f_file)

### 3) run `add_sidecars.py`

In [120]:
tileset = Tileset(Path(qf_dir))
tileset.add_sidecars(f_file, d2m.df.index.name)

note we copied `add_sidecars.py` so you can use it directly from this library

In [44]:
!featherplot add-sidecars --tileset {qf_dir}\
                         --sidecar {f_file} --key {d2m.df.index.name};

```sh
featherplot-py featherplot add-sidecars --help

Usage: featherplot add-sidecars 
[OPTIONS]
--tileset          PATH  Path to the tileset to add sidecars to.
--sidecar          PATH  Path to the new data to add to the tileset.
--key              TEXT  key to use for joining; must exist in both tables
--verbose  -v            Print verbose output.
--help                   Show this message and exit.
```

alternatively you can run the script form wherever you saved it

In [ ]:
!python3 add_sidecars.py --tileset {qf_dir}\
                         --sidecar {f_file} --key {d2m.df.index.name};

### 4) update metadata with directory

In [121]:
meta.keys()

dict_keys(['index', 'n_points', 'embedding', 'sidecars', 'columns_metadata', 'tiles_dir'])

In [122]:
# relative path to tiles
meta['tiles_dir'] = qf_dir.replace(outdir, '')
# full path to tiles
meta['full_path'] = collapse_user(qf_dir)

In [123]:
with open(m_file, 'w') as f:
    f.write(yaml.dump(meta))

### 5) cleanup